# Statistics


**IDEAS**
- SMETANA
    - Find likelyhood that a community member from phyla X is dependent on a community member in phyla Y.
    - Find likelyhood that a community member from phyla X is dependent on a compound A.

- SteadierCOM
    - No statistics?

In [4]:
import pandas as pd
from molmass import Formula
import reframed

In [1]:
import functions.general_functions as general_func

### Load data 

In [2]:
all_mags_paper = general_func.read_allmags_data()

In [5]:
steadiercom_crossfeeding_basic = pd.read_csv("output/steadiercom_results/new_steadiercom_99/steadiercom_crossfeeding_basic.tsv",sep="\t",index_col=(0,1))
steadiercom_crossfeeding_basic

donor   receiver     compound      rate     mass_rate
CD_A 0   CH13_bin_12        NaN       M_ac_e  1.665756  9.835271e-02
     1   CH13_bin_11        NaN       M_ac_e  0.564709  3.334261e-02
     2    CH13_bin_4        NaN       M_ac_e  0.514872  3.040004e-02
     3   CH13_bin_14        NaN       M_ac_e  0.161665  9.545325e-03
     4   CH13_bin_25        NaN       M_ac_e  0.006444  3.804543e-04
...              ...        ...          ...       ...           ...
M_X  75          NaN  CH3_bin_2   M_xylan8_e  0.144231  2.055521e-01
     76          NaN  CH3_bin_2      M_zn2_e  0.000020  1.312212e-06
     77          NaN  CH3_bin_1      M_zn2_e  0.000006  3.790267e-07
     78    CH3_bin_1        NaN    M_gcald_e  0.000011  6.828987e-07
     79          NaN  CH3_bin_1  M_pnto__R_e  0.000010  2.136653e-06

[2749 rows x 5 columns]

In [6]:
steadiercom_crossfeeding_basic = steadiercom_crossfeeding_basic.dropna().copy()
steadiercom_crossfeeding_basic

donor     receiver    compound      rate  mass_rate
CD_A 9   CH13_bin_12  CH13_bin_11   M_acald_e  0.018684   0.000823
     10  CH13_bin_12   CH13_bin_0   M_acald_e  0.000079   0.000003
     11  CH13_bin_12  CH13_bin_13   M_acald_e  0.001600   0.000070
     12   CH13_bin_4  CH13_bin_11   M_acald_e  0.484715   0.021353
     13   CH13_bin_4   CH13_bin_0   M_acald_e  0.002046   0.000090
...              ...          ...         ...       ...        ...
M_X  68    CH3_bin_0    CH3_bin_2     M_ura_e  0.017927   0.002009
     69    CH3_bin_0    CH3_bin_1     M_ura_e  0.007163   0.000803
     70    CH3_bin_1    CH3_bin_2  M_xyl__D_e  2.187086   0.328346
     71    CH3_bin_2    CH3_bin_1  M_xylan4_e  0.281673   0.203252
     72    CH3_bin_2    CH3_bin_0  M_xylan4_e  0.006788   0.004898

[1282 rows x 5 columns]

In [7]:
model_uni = reframed.load_cbmodel("/Users/idunmariaburgos/universal_model_extension/output/universe_bacteria.xml")

### Process data

In [8]:
MAGs_steady_com = set(list(steadiercom_crossfeeding_basic.donor.values)+list(steadiercom_crossfeeding_basic.receiver.values))

MAG2sour_sub_id = pd.read_csv("output/MAG2community_id.tsv",sep="\t",header=None)
MAG2sour_sub_id.columns=["MAG","community_id"]
MAGs_steadycom_dict = {MAG:MAG.split("_")[0]+"-"+MAG.split("_")[1]+"."+MAG.split("_")[2] for MAG in  MAGs_steady_com}



phyla_groups = all_mags_paper[all_mags_paper.index.isin([MAGs_steadycom_dict[MAG] for MAG in MAGs_steady_com])].groupby("Phylum").groups
mag2phyla_dict = {mag:phyla for phyla,mags in phyla_groups.items() for mag in mags}


met2superclass_dict = pd.read_csv("output/met_chebi_class.tsv",sep="\t",index_col=0)["self defined super class"].to_dict()

In [9]:
def phyla_donor(row):
    
    return mag2phyla_dict[MAGs_steadycom_dict[row.donor]]


def phyla_receiver(row):
    
    return mag2phyla_dict[MAGs_steadycom_dict[row.receiver]]

def mmol2mg(row):
    molweight = Formula(model_uni.metabolites[row.compound].metadata["FORMULA"]).mass
    flux_mg = molweight*row.flux
    return flux_mg

In [43]:
def met2metname(met):
    met_name = model_uni.metabolites[met].name
    return met_name

In [64]:
steadiercom_crossfeeding_basic = pd.read_csv("output/steadiercom_results/new_steadiercom_99/steadiercom_crossfeeding_basic.tsv",sep="\t",index_col=(0,1))
steadiercom_crossfeeding_basic

donor   receiver     compound      rate     mass_rate
CD_A 0   CH13_bin_12        NaN       M_ac_e  1.665756  9.835271e-02
     1   CH13_bin_11        NaN       M_ac_e  0.564709  3.334261e-02
     2    CH13_bin_4        NaN       M_ac_e  0.514872  3.040004e-02
     3   CH13_bin_14        NaN       M_ac_e  0.161665  9.545325e-03
     4   CH13_bin_25        NaN       M_ac_e  0.006444  3.804543e-04
...              ...        ...          ...       ...           ...
M_X  75          NaN  CH3_bin_2   M_xylan8_e  0.144231  2.055521e-01
     76          NaN  CH3_bin_2      M_zn2_e  0.000020  1.312212e-06
     77          NaN  CH3_bin_1      M_zn2_e  0.000006  3.790267e-07
     78    CH3_bin_1        NaN    M_gcald_e  0.000011  6.828987e-07
     79          NaN  CH3_bin_1  M_pnto__R_e  0.000010  2.136653e-06

[2749 rows x 5 columns]

In [65]:
steadiercom_crossfeeding_basic = steadiercom_crossfeeding_basic.dropna().copy()
steadiercom_crossfeeding_basic

donor     receiver    compound      rate  mass_rate
CD_A 9   CH13_bin_12  CH13_bin_11   M_acald_e  0.018684   0.000823
     10  CH13_bin_12   CH13_bin_0   M_acald_e  0.000079   0.000003
     11  CH13_bin_12  CH13_bin_13   M_acald_e  0.001600   0.000070
     12   CH13_bin_4  CH13_bin_11   M_acald_e  0.484715   0.021353
     13   CH13_bin_4   CH13_bin_0   M_acald_e  0.002046   0.000090
...              ...          ...         ...       ...        ...
M_X  68    CH3_bin_0    CH3_bin_2     M_ura_e  0.017927   0.002009
     69    CH3_bin_0    CH3_bin_1     M_ura_e  0.007163   0.000803
     70    CH3_bin_1    CH3_bin_2  M_xyl__D_e  2.187086   0.328346
     71    CH3_bin_2    CH3_bin_1  M_xylan4_e  0.281673   0.203252
     72    CH3_bin_2    CH3_bin_0  M_xylan4_e  0.006788   0.004898

[1282 rows x 5 columns]

In [66]:
steadiercom_crossfeeding_basic.loc[:,"phylum_donor"] = steadiercom_crossfeeding_basic.apply(phyla_donor,axis=1).copy()
steadiercom_crossfeeding_basic.loc[:,"phylum_receiver"] = steadiercom_crossfeeding_basic.apply(phyla_receiver,axis=1).copy()
steadiercom_crossfeeding_basic = steadiercom_crossfeeding_basic[steadiercom_crossfeeding_basic.compound.isin(met2superclass_dict.keys())].copy()
steadiercom_crossfeeding_basic.loc[:,"super_class"] = steadiercom_crossfeeding_basic.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadiercom_crossfeeding_basic["compound"].map(met2metname)

steadiercom_crossfeeding_basic.drop("compound",axis=1,inplace=True)

steadiercom_crossfeeding_basic.loc[:,"compound"] = compounds

In [67]:
steadiercom_crossfeeding_basic

donor     receiver      rate  mass_rate    phylum_donor  \
CD_A 9   CH13_bin_12  CH13_bin_11  0.018684   0.000823  Fibrobacterota   
     10  CH13_bin_12   CH13_bin_0  0.000079   0.000003  Fibrobacterota   
     11  CH13_bin_12  CH13_bin_13  0.001600   0.000070  Fibrobacterota   
     12   CH13_bin_4  CH13_bin_11  0.484715   0.021353    Bacteroidota   
     13   CH13_bin_4   CH13_bin_0  0.002046   0.000090    Bacteroidota   
...              ...          ...       ...        ...             ...   
M_X  68    CH3_bin_0    CH3_bin_2  0.017927   0.002009      Firmicutes   
     69    CH3_bin_0    CH3_bin_1  0.007163   0.000803      Firmicutes   
     70    CH3_bin_1    CH3_bin_2  2.187086   0.328346      Firmicutes   
     71    CH3_bin_2    CH3_bin_1  0.281673   0.203252      Firmicutes   
     72    CH3_bin_2    CH3_bin_0  0.006788   0.004898      Firmicutes   

          phylum_receiver                  super_class  \
CD_A 9   Desulfobacterota       alcohols and aldehydes   
     10        Firmicutes       alcohols and aldehydes   
     11     Spirochaetota       alcohols and aldehydes   
     12  Desulfobacterota       alcohols and aldehydes   
     13        Firmicutes       alcohols and aldehydes   
...                   ...                          ...   
M_X  68        Firmicutes  nucleosides and derivatives   
     69        Firmicutes  nucleosides and derivatives   
     70        Firmicutes                simple sugars   
     71        Firmicutes             oligosaccharides   
     72        Firmicutes             oligosaccharides   

                                             compound  
CD_A 9                                   Acetaldehyde  
     10                                  Acetaldehyde  
     11                                  Acetaldehyde  
     12                                  Acetaldehyde  
     13                                  Acetaldehyde  
...                                               ...  
M_X  68                                        Uracil  
     69                                        Uracil  
     70                                      D-Xylose  
     71  Xylan (4 backbone units, 1 glcur side chain)  
     72  Xylan (4 backbone units, 1 glcur side chain)  

[1238 rows x 8 columns]

In [68]:
import scipy.stats as stats

def statistics_function(dependent_variable,independent_variable,metric="flux_mg",metric_thresh=0,pvalue_thresh=0.1):
    
    def find_non_dependent(row):
        return len(phyla_groups[row.name[0]]) - row[metric] #row.flux_mg

    # Get average of each phyla according to each possible value of the independent variable
    
    
    steadiercom_crossfeeding_donor = steadiercom_crossfeeding_basic.loc[:,[dependent_variable,dependent_variable.split("_")[1],independent_variable,metric]].groupby([dependent_variable,dependent_variable.split("_")[1],independent_variable]).mean().copy()


    dependent = steadiercom_crossfeeding_donor[steadiercom_crossfeeding_donor[metric]>metric_thresh].reset_index().groupby([dependent_variable,independent_variable]).count().copy()
    
    
    not_dependent = dependent.apply(find_non_dependent,axis=1)

    
    all_categories =set(not_dependent.index.get_level_values(1))

    for phyla in dependent.index.get_level_values(0):
        for category in all_categories-set(not_dependent.xs(phyla).index):
            not_dependent[(phyla,category)]=len(phyla_groups[phyla]) 
            
    concat_df = pd.concat({"dependent":dependent[metric],"not_dependent":not_dependent},axis=1).fillna(0)


    statistics = {}
    for independent_var in set(concat_df.index.get_level_values(1)):
        # Get the sub_df for the super class
        concat_df_sub = concat_df.xs(independent_var,level=1).copy()

        statistics[independent_var] = {}

        # For each row (each phyla)
        

        for i,row in concat_df_sub.iterrows():
            # Get the data for all other phyla
            other = concat_df_sub.loc[concat_df_sub.index[concat_df_sub.index!=i],:]
            
            # Calculate the Fischer exact statistical 
            odds_ratio, p_value = stats.fisher_exact(pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy())
            
            statistics[independent_var][(i,"odds_ratio")]=odds_ratio
            statistics[independent_var][(i,"p_value")]= p_value 

    statistics_df = pd.DataFrame(statistics)
    

    category_values = statistics_df.xs('p_value', level=1)
    

    values = {}
    for phyla,independent_var in category_values[category_values[category_values<pvalue_thresh].notnull()].stack().index:
        values[phyla,independent_var]= {"p_value":statistics_df.loc[(phyla,"p_value"),independent_var],"odds_ratio":statistics_df.loc[(phyla,"odds_ratio"),independent_var],"# interaction":concat_df.loc[(phyla,independent_var),"dependent"],"# no interaction":concat_df.loc[(phyla,independent_var),"not_dependent"]}
        
    significant = pd.DataFrame(values).transpose()
    if significant.empty==False:
        significant.index.names = (dependent_variable,independent_variable)

    return significant

## Likelyhood that phyla X is receiving compound A

In [69]:
statistics_df = statistics_function("phylum_receiver","super_class",metric="mass_rate",metric_thresh=0,pvalue_thresh=0.1)

statistics_df = statistics_df[statistics_df.odds_ratio>1]
statistics_df[statistics_df["# interaction"]>1]

p_value  odds_ratio  \
phylum_receiver   super_class                                      
Bacteroidota      other                     0.008619    8.181818   
Desulfobacterota  alcohols and aldehydes    0.051516    3.381818   
                  other                     0.095287    2.793651   
Firmicutes        oligopeptides             0.086957         inf   
                  B-vitamins and cofactors  0.086957         inf   
Verrucomicrobiota carbohydrate derivatives  0.062294   12.888889   

                                            # interaction  # no interaction  
phylum_receiver   super_class                                                
Bacteroidota      other                              10.0               2.0  
Desulfobacterota  alcohols and aldehydes             12.0               5.0  
                  other                              11.0               6.0  
Firmicutes        oligopeptides                       2.0              19.0  
                  B-vitamins and cofactors            2.0              19.0  
Verrucomicrobiota carbohydrate derivatives            2.0               1.0

In [70]:
statistics_df = statistics_function("phylum_receiver","compound",metric="mass_rate",metric_thresh=0,pvalue_thresh=0.05)

statistics_df = statistics_df[statistics_df.odds_ratio>1]
statistics_df[statistics_df["# interaction"]>1]

p_value  odds_ratio  # interaction  \
phylum_receiver  compound                                                
Bacteroidota     L-Arginine        0.027329         inf            2.0   
                 Hypoxanthine      0.040502    5.300000            4.0   
                 Indole            0.002067    8.600000            9.0   
Desulfobacterota Quinate           0.007677   10.625000            5.0   
                 Glycolate C2H3O3  0.012422         inf            3.0   
                 Carbon monoxide   0.041902   11.142857            3.0   
                 L-Lactate         0.002911    8.575000            7.0   
Firmicutes       Thymine C5H6N2O2  0.002261   11.750000            7.0   
Spirochaetota    D-Glucose         0.022309    5.666667            5.0   

                                   # no interaction  
phylum_receiver  compound                            
Bacteroidota     L-Arginine                    10.0  
                 Hypoxanthine                   8.0  
                 Indole                         3.0  
Desulfobacterota Quinate                       12.0  
                 Glycolate C2H3O3              14.0  
                 Carbon monoxide               14.0  
                 L-Lactate                     10.0  
Firmicutes       Thymine C5H6N2O2              14.0  
Spirochaetota    D-Glucose                      5.0

#### Flux>10

## Likelyhood that phyla X is exchanging a compound A with another phyla

In [72]:
statistics_df = statistics_function("phylum_donor","super_class",metric="mass_rate",metric_thresh=0,pvalue_thresh=0.1)

statistics_df = statistics_df[statistics_df.odds_ratio>1]
statistics_df[statistics_df["# interaction"]>1]

p_value  odds_ratio  \
phylum_donor      super_class                                         
Bacteroidota      oligopeptides                0.014456   19.000000   
Desulfobacterota  fatty acids                  0.091098    3.769231   
Firmicutes        simple sugars                0.040927    3.545455   
                  nucleosides and derivatives  0.067061    2.750000   
Verrucomicrobiota other                        0.074169         inf   

                                               # interaction  # no interaction  
phylum_donor      super_class                                                   
Bacteroidota      oligopeptides                          3.0               9.0  
Desulfobacterota  fatty acids                            4.0              13.0  
Firmicutes        simple sugars                         10.0              11.0  
                  nucleosides and derivatives           12.0               9.0  
Verrucomicrobiota other                                  3.0               0.0

In [73]:
statistics_df = statistics_function("phylum_donor","super_class",metric="mass_rate",metric_thresh=0,pvalue_thresh=0.05)

statistics_df = statistics_df[statistics_df.odds_ratio>1]
statistics_df[statistics_df["# interaction"]>1]

,,p_value,odds_ratio,# interaction,# no interaction
phylum_donor,super_class,,,,
Bacteroidota,oligopeptides,0.014456,19.000000,3.0,9.0
Firmicutes,simple sugars,0.040927,3.545455,10.0,11.0


## Likelyhood that phyla X is receiving from phyla Y

In [76]:
statistics_df = statistics_function("phylum_receiver","phylum_donor",metric="mass_rate",metric_thresh=0,pvalue_thresh=0.1)
statistics_df = statistics_df[statistics_df.odds_ratio>1]
statistics_df[statistics_df["# interaction"]>1]

p_value  odds_ratio  # interaction  \
phylum_receiver phylum_donor                                              
Bacteroidota    Cloacimonadota      0.060704    4.333333            4.0   
Firmicutes      Thermodesulfobiota  0.077604    8.000000            3.0   
                Actinobacteriota    0.086957         inf            2.0   

                                    # no interaction  
phylum_receiver phylum_donor                          
Bacteroidota    Cloacimonadota                   8.0  
Firmicutes      Thermodesulfobiota              18.0  
                Actinobacteriota                19.0

In [77]:
statistics_df = statistics_function("phylum_receiver","phylum_donor",metric="mass_rate",metric_thresh=0,pvalue_thresh=0.05)
statistics_df = statistics_df[statistics_df.odds_ratio>1]
statistics_df[statistics_df["# interaction"]>1]

,,p_value,odds_ratio,# interaction,# no interaction
phylum_receiver,phylum_donor,,,,
